In [1]:
#implemented ideas from https://towardsdatascience.com/yet-another-twitter-sentiment-analysis-part-1-tackling-class-imbalance-4d7a7f717d44

In [2]:
import pandas as pd
import numpy as np
import json
import re
import string
import nltk
import spacy
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS


In [3]:
nltk.download('words')
eng_words=set(nltk.corpus.words.words())
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords=stopwords.words('english')
stopwords.extend(['book','story','read','reading','really','one'])

[nltk_data] Downloading package words to /home/ramya/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ramya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
filepath='E:/Ramya/brushUps/Basics/ML using sklearn/Projects/Sentiment_analysis/data/goodreads_reviews_children.json.gz'
with open(filepath) as f:
    lines=f.read().splitlines()

In [5]:
data_inter=pd.DataFrame(lines)
data_inter.columns=['json_element']

In [7]:
data_inter['json_element'].apply(json.loads)

0         {'user_id': '8842281e1d1347389f2ab93d60773d4d'...
1         {'user_id': '8842281e1d1347389f2ab93d60773d4d'...
2         {'user_id': '8842281e1d1347389f2ab93d60773d4d'...
3         {'user_id': '8842281e1d1347389f2ab93d60773d4d'...
4         {'user_id': '8842281e1d1347389f2ab93d60773d4d'...
                                ...                        
734635    {'user_id': '480ac8b4df39533a6c7fe6ae109aa56a'...
734636    {'user_id': 'afcd096765a8c354be0706ab1324999b'...
734637    {'user_id': 'd94c83867337514c94738b57a1d19677'...
734638    {'user_id': '48a10d546ba07520ce194dc18e687be4'...
734639    {'user_id': 'da6f9516279ed671027308abd21318bd'...
Name: json_element, Length: 734640, dtype: object

In [8]:
data=pd.json_normalize(data_inter['json_element'].apply(json.loads))

In [9]:
reviews_data=data[['book_id','rating','review_text']]

In [10]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                   "can't've": "cannot have", "'cause": "because", "could've": "could have", 
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
                   "he'll've": "he will have", "he's": "he is", "how'd": "how did", 
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                   "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
                   "i'll've": "i will have","i'm": "i am", "i've": "i have", 
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                   "it'll": "it will", "it'll've": "it will have","it's": "it is", 
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                   "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is", 
                   "there'd": "there would", "there'd've": "there would have","there's": "there is", 
                       "here's": "here is",
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                   "they'll've": "they will have", "they're": "they are", "they've": "they have", 
                   "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                   "we're": "we are", "we've": "we have", "weren't": "were not", 
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                   "what's": "what is", "what've": "what have", "when's": "when is", 
                   "when've": "when have", "where'd": "where did", "where's": "where is", 
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                   "who's": "who is", "who've": "who have", "why's": "why is", 
                   "why've": "why have", "will've": "will have", "won't": "will not", 
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                   "you'll've": "you will have", "you're": "you are", "you've": "you have" } 

In [11]:
def reviewsTextCleaning(review):
    #expanding words
    review=' '.join([contraction_mapping[word] if word in contraction_mapping.keys() else word for word in review.split()])
    #removing @mentions
    review=re.sub(r'@[A-Za-z0-9]+',' ',review)
    review=review.lower()
    review=review.strip()
    review=review.replace('\n','')
    review=review.replace('\t','')
    #converting apostrophes to single quote
    review=review.replace("’", "'")
    #removing punctuations
    review=review.translate(str.maketrans('','',string.punctuation))
    #removinf digits
    review=re.sub(r'\d+','',review)
    #removing urls
    review=re.sub(r'^https?:\/\/.*[\r\n]*','',review)
    #removing tags
    remove_tags=re.compile('<.*?>')
    review=re.sub(remove_tags,'',review) 
    #lemmatization
    nlp=spacy.load('en_core_web_sm',disable=['parser','ner'])
    review=' '.join([token.lemma_ for token in nlp(review)])
    #ignoring gibberish words
    review=' '.join(w for w in nltk.wordpunct_tokenize(review) if w.lower() in eng_words or not w.isalpha())

    return review                                                                                                                                                                                                                                                                                                                                                   